In [2]:
pip install z3-solver

     |████████████████████████████████| 30.5MB 147kB/s 


In [3]:
from z3 import *

In [121]:
class Kripke:
    def __init__(self, states, init, Trans):
        self.states = states
        self.Trans = Trans
        self.init = init

    def state(self, n):
        return [Bool(var + str(n)) for var in self.states]
    
    def initConstraint(self, state):
        return And([state[i] == self.init[self.states[i]] for i in range(len(self.states))])

    def pathConstraint(self, pathStates):
        constraint = []
        for i in range(len(pathStates)-1):
            constraint += [self.Trans(pathStates[i], pathStates[i+1])]
        return And(self.initConstraint(pathStates[0]), And(constraint))

    def LTLConstraint(self, i, k, formula, pathStates):
        if list(formula.keys())[0] == 'p':
            return formula['p'](pathStates[i])
        elif list(formula.keys())[0] == '!p':
            return Not(formula['!p'](pathStates[i]))
        elif list(formula.keys())[0] == 'Or':
            return Or(self.LTLConstraint(i, k, formula['Or'][0], pathStates), self.LTLConstraint(i, k, formula['Or'][1], pathStates))
        elif list(formula.keys())[0] == 'And':
            return And(self.LTLConstraint(i, k, formula['And'][0], pathStates), self.LTLConstraint(i, k, formula['And'][1], pathStates))
        elif list(formula.keys())[0] == 'G':
            return False
        elif list(formula.keys())[0] == 'F':
            return Or([self.LTLConstraint(j, k, formula['F'], pathStates) for j in range(i, k+1)])
        elif list(formula.keys())[0] == 'X':
            if i < k:
                return self.LTLConstraint(i+1, k, formula['X'], pathStates)
            else:
                return False
        elif list(formula.keys())[0] == 'U':
            return Or([And(self.LTLConstraint(j, k, formula['U'][1], pathStates), And([self.LTLConstraint(n, k, formula['U'][0], pathStates) for n in range(i, j)])) for j in range(i, k+1)])
        elif list(formula.keys())[0] == 'R':
            return Or([And(self.LTLConstraint(j, k, formula['U'][0], pathStates), And([self.LTLConstraint(n, k, formula['U'][1], pathStates) for n in range(i, j+1)])) for j in range(i, k+1)])

    def LTLConstraintLoop(self, i, k, l, formula, pathStates):
        if list(formula.keys())[0] == 'p':
            return formula['p'](pathStates[i])
        elif list(formula.keys())[0] == '!p':
            return Not(formula['!p'](pathStates[i]))
        elif list(formula.keys())[0] == 'Or':
            return Or(self.LTLConstraintLoop(i, k, l, formula['Or'][0], pathStates), self.LTLConstraintLoop(i, k, l, formula['Or'][1], pathStates))
        elif list(formula.keys())[0] == 'And':
            return And(self.LTLConstraintLoop(i, k, l, formula['And'][0], pathStates), self.LTLConstraintLoop(i, k, l, formula['And'][1], pathStates))
        elif list(formula.keys())[0] == 'G':
            return And([self.LTLConstraintLoop(j, k, l, formula['G'], pathStates) for j in range(min(i, l), k+1)])
        elif list(formula.keys())[0] == 'F':
            return Or([self.LTLConstraintLoop(j, k, l, formula['F'], pathStates) for j in range(min(i, l), k+1)])
        elif list(formula.keys())[0] == 'X':
            if i < k-1:
                return self.LTLConstraintLoop(i+1, k, l, formula['X'], pathStates)
            elif i == k-1:
                return self.LTLConstraintLoop(l, k, l, formula['X'], pathStates)
        elif list(formula.keys())[0] == 'U':
            constraint1 = Or([And(self.LTLConstraintLoop(j, k, l, formula['U'][1], pathStates), And([self.LTLConstraintLoop(n, k, l, formula['U'][0], pathStates) for n in range(i, j)])) for j in range(i, k+1)])
            constraint2 = Or([And(self.LTLConstraintLoop(j, k, l, formula['U'][1], pathStates), And([self.LTLConstraintLoop(n, k, l, formula['U'][0], pathStates) for n in range(i, k+1)]), And([self.LTLConstraintLoop(n, k, l, formula['U'][0], pathStates) for n in range(l, j)])) for j in range(l, i)])
            return Or(constraint1, constraint2)
        elif list(formula.keys())[0] == 'R':
            constraint1 = And([self.LTLConstraintLoop(j, k, l, formula['R'][1], pathStates) for j in range(min(i, l), k+1)])
            constraint2 = Or([And(self.LTLConstraintLoop(j, k, l, formula['U'][0], pathStates), And([self.LTLConstraintLoop(n, k, l, formula['U'][1], pathStates) for n in range(i, j+1)])) for j in range(i, k+1)])
            constraint3 = Or([And(self.LTLConstraintLoop(j, k, l, formula['U'][0], pathStates), And([self.LTLConstraintLoop(n, k, l, formula['U'][1], pathStates) for n in range(i, k+1)]), And([self.LTLConstraintLoop(n, k, l, formula['U'][1], pathStates) for n in range(l, j)])) for j in range(l, i)])
            return Or(constraint1, constraint2, constraint3)

    def Loop(self, l, k, pathStates):
        return self.Trans(pathStates[k], pathStates[l])

    def KmodelCheck(self, k, formula):
        pathStates = []
        for j in range(k+1):
            pathStates += [self.state(j)]
        pathConstraint = self.pathConstraint(pathStates)
        KLoopConstraint = Or([self.Loop(l, k, pathStates) for l in range(k+1)])
        LTLConstraint = And(Not(KLoopConstraint), self.LTLConstraint(0, k, formula, pathStates))
        LTLConstraintLoop = Or([And(self.Loop(l, k, pathStates), self.LTLConstraintLoop(0, k, l, formula, pathStates)) for l in range(k+1)])
        return And(pathConstraint, Or(LTLConstraint, LTLConstraintLoop))

In [129]:
def Trans(curr, next):
    return Or(And(Not(curr[0]), Not(curr[1]), Not(next[0]), next[1]),
       And(Not(curr[0]), curr[1], Not(next[0]), next[1]),
       And(Not(curr[0]), curr[1], next[0], Not(next[1])),
       And(Not(curr[0]), curr[1], next[0], next[1]),
       And(curr[0], Not(curr[1]), next[0], next[1]),
       And(curr[0], curr[1], next[0], next[1]))

def p(state):
    return Not(state[0])
def q(state):
    return state[1]

In [130]:
k = Kripke(['x', 'y'], {'x': False, 'y': False}, Trans)
s = z3.Optimize()

f = {'G': {'p': p}}
s.add(k.KmodelCheck(7, f))
print(s)
print(s.check())
print(s.model())

(declare-fun x7 () Bool)
(declare-fun x6 () Bool)
(declare-fun x5 () Bool)
(declare-fun x4 () Bool)
(declare-fun x3 () Bool)
(declare-fun x2 () Bool)
(declare-fun x1 () Bool)
(declare-fun x0 () Bool)
(declare-fun y7 () Bool)
(declare-fun y6 () Bool)
(declare-fun y5 () Bool)
(declare-fun y4 () Bool)
(declare-fun y3 () Bool)
(declare-fun y2 () Bool)
(declare-fun y1 () Bool)
(declare-fun y0 () Bool)
(assert (let ((a!1 (or (and (not x7) (not y7) (not x0) y0)
               (and (not x7) y7 (not x0) y0)
               (and (not x7) y7 x0 (not y0))
               (and (not x7) y7 x0 y0)
               (and x7 (not y7) x0 y0)
               (and x7 y7 x0 y0)))
      (a!2 (or (and (not x7) (not y7) (not x1) y1)
               (and (not x7) y7 (not x1) y1)
               (and (not x7) y7 x1 (not y1))
               (and (not x7) y7 x1 y1)
               (and x7 (not y7) x1 y1)
               (and x7 y7 x1 y1)))
      (a!3 (or (and (not x7) (not y7) (not x2) y2)
               (and (not x7) y7 (